In [47]:
import TopDownHockey_Scraper.TopDownHockey_NHL_Scraper as tdhnhlscrape
import TopDownHockey_Scraper.TopDownHockey_EliteProspects_Scraper as tdhepscrape
from nhlpy import NHLClient
import pandas as pd
from datetime import date
import statsmodels.api as sm
import numpy as np
from sklearn.linear_model import LogisticRegression
from lets_plot import *
LetsPlot.setup_html()
from pins import board_s3

In [48]:
scrape = True # Set to True to scrape data, False to use existing data

#This will be the file name where data is stored in our git repo
filename = "prospects_2020_to_2025.xlsx"

In [49]:
#Get OHL Player Data (stats, build, etc)

if scrape == True:

    #Can't include all years because of backend API issues in some years
    years = ["2019-2020", "2020-2021", "2021-2022", "2022-2023", "2023-2024", "2024-2025"]

    aggregated_output = pd.DataFrame()

    for year in years:
        df = tdhepscrape.get_skaters(("ohl"), (year))

        #GET PLAYER INFO
        info = tdhepscrape.get_player_information(df)
        
        #GET RID OF DEFENCEMEN
        df = df[~df['player'].str.contains(r'\(([^)]*D[^)]*)\)', regex=True)]

        #GET RID OF PLAYER POSITIONS FROM NAMES
        df['player'] = df['player'].str.replace(r'\s*\([^)]*\)', '', regex=True)

        #ADD YEAR TO DF
        df.insert(0, "year", year)

        #JOIN PLAYER BIO WITH STATS
        year_output = pd.merge(df[["year", "player", "gp", "g", "a", "tp"]], info[["player", "dob", "height", "weight", "shoots"]], on='player', how='inner')

        #ADD CURRENT YEAR PROSPECTS TO AGGREGATED DF
        aggregated_output = pd.concat([aggregated_output, year_output])


Your scrape request is skater data from the following league:
ohl
In the following season:
2019-2020
Beginning scrape of ohl skater data from 2019-2020.
Successfully scraped all ohl skater data from 2019-2020.
Scraping ohl data is complete. You scraped skater data from 2019-2020.
Completed scraping skater data from the following league:
ohl
Over the following season:
2019-2020
Beginning scrape for 465 players.
Marko Sikic scraped! That's 1 down! Only 464 left to go!
Brett Neumann scraped! That's 2 down! Only 463 left to go!
Hayden Fowler scraped! That's 3 down! Only 462 left to go!
Luke Boka scraped! That's 4 down! Only 461 left to go!
Nick DeGrazia scraped! That's 5 down! Only 460 left to go!
Calvin Martin scraped! That's 6 down! Only 459 left to go!
Cameron Morton scraped! That's 7 down! Only 458 left to go!
Wyatt Johnston scraped! That's 8 down! Only 457 left to go!
Dylan Robinson scraped! That's 9 down! Only 456 left to go!
Frank Jenkins scraped! That's 10 down! Only 455 left to go

In [50]:
if scrape == True:
    years = list(range(2020, 2026))

    draftyears = pd.DataFrame()

    for year in years:
        df_list = pd.read_html(f"https://www.hockey-reference.com/draft/NHL_{year}_entry.html", match="Round")

        players_drafted = df_list[0]

        players_drafted

        #Let's get rid of the top header that isnt really used
        players_drafted.columns = players_drafted.columns.get_level_values(1)

        players_drafted["draft_year"] = year
        players_drafted = players_drafted[["draft_year", "Player"]]
        players_drafted =players_drafted.rename(columns={"draft_year": "player_draft_year", "Player": "player"})
        draftyears = pd.concat([draftyears, players_drafted])

    print(draftyears)    

     player_draft_year             player
0                 2020  Alexis Lafreniere
1                 2020    Quinton Byfield
2                 2020        Tim Stützle
3                 2020      Lucas Raymond
4                 2020     Jake Sanderson
..                 ...                ...
231               2025     Jacob Cloutier
232               2025       Filip Ekberg
233               2025   Charlie Paquette
234               2025         Aidan Park
235               2025       Yegor Midlak

[1409 rows x 2 columns]


In [51]:
#Getting games played for all NHL players
if scrape == True:
    nhl_gp = pd.DataFrame()
    pages = list(range(1, 80))

    for page in pages:
        df_list = pd.read_html(f"https://www.eliteprospects.com/league/nhl/stats/all-time?page={page}")
        page_stats = df_list[2]
        page_stats = page_stats[["Player", "GP"]]
        nhl_gp = pd.concat([nhl_gp, page_stats])

    nhl_gp = nhl_gp.rename(columns={"Player": "player", "GP": "nhl_gp"})

    #GET RID OF PLAYER POSITIONS FROM NAMES
    nhl_gp['player'] = nhl_gp['player'].str.replace(r'\s*\([^)]*\)', '', regex=True)

    # Replacing NA and "-" values with 0
    nhl_gp["nhl_gp"] = nhl_gp["nhl_gp"].fillna(0)
    nhl_gp["nhl_gp"] = nhl_gp["nhl_gp"].replace("-", 0)

    nhl_gp

In [52]:
if scrape == True:
    aggregated_output["gp"] = aggregated_output["gp"].replace("-", 0)
    aggregated_output["gp"] = aggregated_output["gp"].astype(int)
    aggregated_output["g"] = aggregated_output["g"].replace("-", 0)
    aggregated_output["g"] = aggregated_output["g"].astype(int)
    aggregated_output["a"] = aggregated_output["a"].replace("-", 0)
    aggregated_output["a"] = aggregated_output["a"].astype(int)
    aggregated_output["tp"] = aggregated_output["tp"].replace("-", 0)
    aggregated_output["tp"] = aggregated_output["tp"].astype(int)

In [53]:
if scrape == True:
    aggregated_output["draft_year"] = aggregated_output["year"].str[5:]
    aggregated_output["draft_year"] = aggregated_output["draft_year"].astype(int)
    aggregated_output

In [54]:
#Join GP, draft year onto OHL player data
if scrape == True:
    #MAKE ALL PLAYER NAMES UPPERCASE (TO MAKE JOINING TABLES NON CASE SENSITIVE)
    aggregated_output['player'] = aggregated_output['player'].str.upper()
    draftyears['player'] = draftyears['player'].str.upper()
    # nhl_gp['player'] = nhl_gp['player'].str.upper()


    #Filter ohl stats for only drafted players' draft year stats - 
    #This will get rid of a) undrafted players, and b) drafted players non-draft year stats
    df = pd.merge(aggregated_output, draftyears, left_on=['player', 'draft_year'], right_on=['player', 'player_draft_year'], how='inner')

    #Can get rid of one of the draft year columns - don't need both
    df = df[["year", "player", "gp", "g", "a", "tp", "dob",	"height", "weight",	"shoots", 'draft_year']]

    #Join players' games played - if player gp not found, assume it to be 0.
    df = pd.merge(df, nhl_gp, left_on=['player'], right_on=['player'], how='left')
    df["nhl_gp"] = df["nhl_gp"].fillna(0)
    df["nhl_gp"] = df["nhl_gp"].astype(int)

    df.to_excel(filename, index=False)

In [55]:
df = pd.read_excel(filename)

#Changing the weight to a numerical variate in kg
df["weight_kg"] = df["weight"].apply(lambda x: x[:x.find(" ")])
df["weight_kg"] = df["weight_kg"].astype(int)
df.dtypes

#Can get rid of intermediate columns
df = df.drop(["weight"], axis = 1)

#Renaming height column to height_cm for clarity
df = df.rename(columns = {"height": "height_cm"})
df

,year,player,gp,g,a,tp,dob,height_cm,shoots,draft_year,nhl_gp,weight_kg
0,2019-2020,MARCO ROSSI,56,39,81,120,2001-09-23,176,L,2020,0,87
1,2019-2020,COLE PERFETTI,61,37,74,111,2002-01-01,180,L,2020,0,84
2,2019-2020,JACK QUINN,62,52,37,89,2001-09-19,185,R,2020,0,84
3,2019-2020,QUINTON BYFIELD,45,32,50,82,2002-08-19,195,L,2020,0,102
4,2019-2020,TYSON FOERSTER,62,36,44,80,2002-01-18,188,R,2020,0,97
...,...,...,...,...,...,...,...,...,...,...,...,...
111,2024-2025,HARRY NANSI,67,7,16,23,2007-09-10,190,R,2025,0,84
112,2024-2025,JORDAN CHARRON,48,10,11,21,2007-06-21,185,L,2025,0,86
113,2024-2025,MATTHEW HLACAR,43,7,1,8,2006-05-31,191,L,2025,0,91
114,2024-2025,KIEREN DERVIN,10,1,2,3,2007-03-31,186,L,2025,0,83


In [56]:
#Getting the age of the player at the time of draft (for simplicity, we will assume draft to be on June 30 for all years)
df["draft_date"] = df["draft_year"].astype(str) + '-06-30'
df["draft_date"] = pd.to_datetime(df["draft_date"])
df["dob"] = pd.to_datetime(df["dob"])
df["age_days"] = (df["draft_date"] - df["dob"])
df["age_days"] = df["age_days"].dt.days

#Can get rid of intermediate columns
df = df.drop(["draft_date", "draft_year", "dob"], axis = 1)

#Adding columns for goals/g and points/g
df["gpg"] = df["g"] / df["gp"]
df["ppg"] = df["tp"] / df["gp"]
df

,year,player,gp,g,a,tp,height_cm,shoots,nhl_gp,weight_kg,age_days,gpg,ppg
0,2019-2020,MARCO ROSSI,56,39,81,120,176,L,0,87,6855,0.696429,2.142857
1,2019-2020,COLE PERFETTI,61,37,74,111,180,L,0,84,6755,0.606557,1.819672
2,2019-2020,JACK QUINN,62,52,37,89,185,R,0,84,6859,0.838710,1.435484
3,2019-2020,QUINTON BYFIELD,45,32,50,82,195,L,0,102,6525,0.711111,1.822222
4,2019-2020,TYSON FOERSTER,62,36,44,80,188,R,0,97,6738,0.580645,1.290323
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,2024-2025,HARRY NANSI,67,7,16,23,190,R,0,84,6503,0.104478,0.343284
112,2024-2025,JORDAN CHARRON,48,10,11,21,185,L,0,86,6584,0.208333,0.437500
113,2024-2025,MATTHEW HLACAR,43,7,1,8,191,L,0,91,6970,0.162791,0.186047
114,2024-2025,KIEREN DERVIN,10,1,2,3,186,L,0,83,6666,0.100000,0.300000


In [57]:
#Storing Other Data used by app / report in s3 bucket:
board = board_s3("devakshah-stat468-models", allow_pickle_read=True)
board.pin_write(df, name="prospects_2020_to_2025_data", type="csv")

Writing pin:
Name: 'prospects_2020_to_2025_data'
Version: 20250806T133431Z-50ed5


Meta(title='prospects_2020_to_2025_data: a pinned 116 x 13 DataFrame', description=None, created='20250806T133431Z', pin_hash='50ed5985308e9aab', file='prospects_2020_to_2025_data.csv', file_size=9918, type='csv', api_version=1, version=Version(created=datetime.datetime(2025, 8, 6, 13, 34, 31, 428731), hash='50ed5985308e9aab'), tags=None, name='prospects_2020_to_2025_data', user={}, local={})